In [43]:
import pandas as pd
import re
from collections import defaultdict
import spacy
from glob import glob
import unidecode
import os
import string
import numpy as np
import pickle as pkl
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import pickle as pkl

import warnings
warnings.filterwarnings("ignore")

# Text pre-processing

In [29]:
PATH = '../'

PROCESSED_DATA_PATH = os.path.join(PATH, 'data/processed/')

DF_FAKE_PATH = glob(PROCESSED_DATA_PATH + '*')[0]
DF_LEGIT_PATH = glob(PROCESSED_DATA_PATH + '*')[1]

df_fake = pkl.load(open(DF_FAKE_PATH, 'rb'))

df_legit = pkl.load(open(DF_LEGIT_PATH, 'rb'))

df = pd.concat((df_fake, df_legit), axis=0)

df.rename({'TEXT_CLEAN': 'X', 'FAKE': 'y'}, axis=1, inplace=True)

df.head()

,DATE,y,X,TITLE_CLEAN
0,03/08/2019,True,vamos assinar peticao cassacao mandato bolsona...,peticao impeachment bolsonaro precisa milhoes ...
1,04/08/2019,True,lula vitima golpe politico merece estar preso ...,peticao lula livre contribui liberdade preside...
2,05/08/2019,True,professor contou dilma matou mario kozel filho...,mario kozel filho assassinado dilma tiros
3,03/08/2019,True,vergonha presidente oab mentiu pai morto milit...,felipe santa cruz presidente oab mentiu sobre ...
4,06/08/2019,True,partido diabolico bandidos abrindo buracos est...,esquerda abrindo buracos estradas nordeste con...


In [30]:
df.shape

(16956, 4)

# Training word2vec model

In [31]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

## Join terms that are commonly found together

e.g.: belo_horizonte

In [32]:
sent = [text.split() for text in df['X']]
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [33]:
df['X_phrased'] = [phrases[s] for s in sent]

In [68]:
w2v_model = Word2Vec(df['X_phrased'],
                    min_count=5,
                    window=7,
                    size=100,
                    workers=3,
                    iter=30)

In [78]:
wrd = 'rei'
print("Palavra: {}".format(wrd))
print("Palavras semelhantes:")
if wrd in w2v_model:
    for item in w2v_model.wv.most_similar(wrd):
        print('- {}'.format(item[0]))

Palavra: rei
Palavras semelhantes:
- monarca
- dom_juan
- rei_espanha
- dom_felipe
- trono
- zarzuela
- rei_felipe
- rei_juan
- akihito
- imperador


In [5]:
from tqdm import tqdm_notebook

In [43]:
all_df = pd.DataFrame()

for text in tqdm_notebook(df['X_phrased']):
    aux = pd.DataFrame()
    for word in text:
        try:
            word_vec = w2v_model[word]
            aux = aux.append(pd.Series(word_vec), ignore_index=True)
        except:
            pass
        doc_vec = aux.mean()
    all_df = all_df.append(doc_vec, ignore_index=True)

c:\users\hidea\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

In [39]:
# pkl.dump(all_df, open('../data/processed/text_w2v.pkl', 'wb'))

all_df = pkl.load(open('../data/processed/text_w2v.pkl', 'rb'))

In [40]:
rus = RandomUnderSampler(random_state=42)
resampled_X, resampled_y = rus.fit_resample(all_df, df['y'])

ValueError: Found input variables with inconsistent numbers of samples: [16858, 16956]

In [17]:
train_x, test_x, train_y, test_y = train_test_split(resampled_X,
                                                   resampled_y,
                                                   test_size = 0.2,
                                                   random_state = 1)

In [18]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((1844, 50), (462, 50), (1844,), (462,))

In [23]:
model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(all_df)

In [37]:
print(classification_report(df['y'], test_pred))

ValueError: Found input variables with inconsistent numbers of samples: [16956, 16858]

In [36]:
print(confusion_matrix(df['y'], test_pred))

ValueError: Found input variables with inconsistent numbers of samples: [16956, 16858]